In [ ]:
#подключем нужные нам библеотеки
import sqlite3
import requests
from bs4 import BeautifulSoup
#создаём функцию, которая будет приводить к числу, передаваемою ей в качестве параметра, строку
def find_number(text):
    #проверяем что символ - цифра, и объединяем его, с возвращаемым значением, через пустую строку
    return int("0" + "".join(i for i in text if i.isdigit()))
#создаём всмомогательную функцию, которая будет возвращать
#все необходимые данные с детальной старницы
def find_data (link):
    #получаем контент с сайта beru.ru, с адреса, который представляет собой страницу,
    #на которой в поле поиска  товаров выбран "Саратов", + ссылка на детальную страницу товара 
    r = requests.get("https://video.ittensive.com/data/018-python-advanced/beru.ru/" + link)
    #преобразуем контент в объект
    html = BeautifulSoup(r.content)
    #получаем название холодильника, оно храниться в первом теге h1
    #который имеет класс "_3TfWusA7bt", преобразуем полученную информацию в текст
    title = html.find("h1", {"class": "_3TfWusA7bt"}).get_text()
    #находим цену холодильника
    #получаем первый тег спан, который имеет "data-tid", равный "c3eaad93"
    price = find_number(html.find("span", {"data-tid": "c3eaad93"}).get_text())
    #получаем все теги "sapn", которые имеют класс "sapn", преобразуем информацию в текст
    #именно в этих тегах хранятся нужные нам, параметры холодильника
    tags = html.find_all("span", {"class": "_112Tad-7AP"})
    #создаём переменные для хранения параметров, на тот случай,
    #если при паргсинге они вдруг не будут определены
    #ширины
    width = 0
    #глубины
    depth = 0
    #высоты
    height = 0
    #общий объем
    volume = 0
    #объем холодильной камеры
    freezer = 0 
    #обходим циклом данные в которых храниться информация о параметрах холодильника
    for tag in tags:
        #преобразуем эту информацию в текст
        tag = tag.get_text() 
        #если в теге присутствует данная строка "ШхВхГ"
        if tag.find("ШхВхГ") > -1:
            #то эту строку мы разделяем по двоеточию,
            #затем вторую часть данной строки (индекс [1]) мы ещё раз делим по символу "х"
            dims = tag.split(":")[1].split("х")
            #первое значение в получившейся строке, является значением ширины
            #получаем его, и преобразуем к типу данных float
            width = float(dims[0])
            #глубина, второе заначение
            depth = float(dims[1])
            #высота третье
            #чтобы избваиться от сантимктров, делим строку по пробелу, и берём её левую часть
            height = float(dims[2].split(" ")[0])
        #если в теге присутствет данная строка "общий объем"
        if tag.find("общий объем") > -1:
            #если в теге присутствует данная строка "общий объем", 
            volume = find_number(tag)
        #если в теге присутствет данная строка "объем холодильной камеры"
        if tag.find("объем холодильной камеры") > -1:
            #то мы присваеваем переменной freezer, значение этой строки
            freezer = find_number(tag)
    #фунцкия возвращает ссылку детальной страницы холодильника, название,
    #цену, ширину, глубину, высоту, общий объём и объём холодильной камеры
    return [link, title, price, width, depth, height, volume, freezer]
#в созданную, для получения данных со страницы функуию
#передаём одну из детальных страниц, модели холодильника
find_data("kholodilnik-saratov-452-ksh-120.html")

#получаем контент со страницы, на которой хранятся все ссылки на детальные страницы холодильника
r = requests.get("https://video.ittensive.com/data/018-python-advanced/beru.ru/")
#преобразуем контент в объект
html = BeautifulSoup(r.content)
#получаем все данные со из ссылок, с атрибутом "а", с классом "grid_snippet_react-link"
links = html.find_all("a", {"class": "grid-snippet__react-link"})
#создаём массив, для того чтобы составить список ссылок детальных страниц
data  = []
#обходим циклом набор полученных ссылок
for link in links:
    #если у ссылки есть атрибут href и в её текстовом представлении есть значение "Сартов"
    if link["href"] and link.get_text().find("Саратов") > -1:
        #в массив data добавляем результат работы функции find_data()
        data.append(find_data(link["href"]))
#подключаемся к базе данных "data.db3"
conn = sqlite3.connect("E:/programs/sqlite/data.db3")
db = conn.cursor()
#в базе данных создаём таблицу "beru_goods", полями id (первичный ключ,  автоинкрементен, не null),
#url (text), title (text), price (целочисленное значение, по умолчанию равно 0),
#width (дробное знчение, по умолчанию равно 0.0),  depth (дробное знчение, по умолчанию равно 0.0),
#depth (дробное знчение, по умолчанию равно 0.0), height  (дробное знчение, по умолчанию равно 0.0),
# volume (целочисленное значение, по умолчанию равно 0), freezer price (целочисленное значение, по умолчанию равно 0)
db.execute("""CREATE TABLE beru_goods
            (id INTEGER PRIMARY KEY AUTOINCREMENT not null,
            url text,
            title text default '',
            price INTEGER default 0,
            width FLOAT default 0.0,
            depth FLOAT default 0.0,
            height FLOAT default 0.0,
            volume INTEGER default 0,
            freezer INTEGER default 0)""")
#выполняем коммит результата
conn.commit()
#в базу данных, с помощью запроса INSERT мы передаём данные
#в таблицу beru_goods, в качестве передаваемых параметров указываем,
#массив data, используем executemany, для того, чтобы добавить множество строк одновременно
db.executemany("""INSERT INTO beru_goods (url, title, price, width, depth, height, volume, freezer)
           VALUES (?, ?, ?, ?, ?, ?, ?, ?)""", data)
conn.commit()
#выводим все значения из созданной базы данных
print(db.execute("SELECT * FROM beru_goods").fetchall())
#закрываем соединения с базой данных
db.close()
